#Libraries

In [1]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import SimpleRNN, Dense
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tf.random.set_seed(42)
np.random.seed(42)

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Tesi/Agrimonia_scaled_Bertonico.csv")
pd.set_option('display.max_columns', None)

df.head()

,Unnamed: 0,AQ_pm10,AQ_nh3,AQ_nox,AQ_no2,WE_temp_2m,WE_wind_speed_10m_mean,WE_wind_speed_10m_max,WE_tot_precipitation,WE_precipitation_t,WE_surface_pressure,WE_solar_radiation,WE_rh_min,WE_rh_mean,WE_rh_max,WE_wind_speed_100m_mean,WE_wind_speed_100m_max,WE_blh_layer_max,WE_blh_layer_min,EM_nh3_livestock_mm,EM_nh3_agr_soils,EM_nh3_agr_waste_burn,EM_nh3_sum,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LI_pigs_v2,LI_bovine_v2,LA_hvi,LA_lvi,LA_land_use,LA_soil_use,wind_dir_10m_N,wind_dir_100m_N,wind_dir_10m_S,wind_dir_100m_S,wind_dir_10m_E,wind_dir_100m_E,wind_dir_10m_O,wind_dir_100m_O,wind_dir_10m_NE,wind_dir_100m_NE,wind_dir_10m_NO,wind_dir_100m_NO,wind_dir_10m_SO,wind_dir_100m_SO,wind_dir_10m_SE,wind_dir_100m_SE
0,4384,0.361111,0.187541,0.169904,0.282810,0.133232,0.085441,0.112459,0.000011,0.000000,0.710983,0.209027,0.669497,0.840646,0.971037,0.060817,0.074656,0.061210,0.005039,0.003703,0.005021,0.314432,0.004290,0.505559,0.690451,0.892958,0.483245,1.000000,0.000000,0.384937,0.001179,0.003807,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4385,0.270833,0.072131,0.135456,0.302741,0.096618,0.170360,0.175662,0.214557,0.166667,0.556840,0.016265,0.923586,0.964518,0.989766,0.176173,0.193587,0.037907,0.051926,0.004526,0.005889,0.304480,0.005208,0.507314,0.686191,0.890141,0.486185,0.989011,0.007823,0.380753,0.001179,0.003807,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4386,0.180556,0.049836,0.127760,0.293174,0.136445,0.267975,0.301458,0.040182,0.166667,0.383430,0.122342,0.840430,0.921590,0.989959,0.293446,0.352442,0.062212,0.141779,0.005226,0.006711,0.293849,0.006062,0.509070,0.682286,0.887655,0.489124,0.989011,0.014342,0.376569,0.001179,0.003807,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4387,0.201389,0.047213,0.108386,0.222380,0.125409,0.143309,0.300480,0.058296,0.166667,0.200385,0.057699,0.845923,0.897988,0.983588,0.172888,0.343952,0.059602,0.017870,0.005925,0.007491,0.282766,0.006851,0.510825,0.678026,0.885501,0.492063,0.978022,0.020860,0.376569,0.001179,0.002538,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4388,0.236111,0.075410,0.162633,0.273722,0.115206,0.098820,0.099011,0.018412,0.166667,0.211946,0.043669,0.894743,0.960558,0.986098,0.120138,0.130859,0.029551,0.022666,0.006542,0.008234,0.271230,0.007577,0.511995,0.673766,0.883513,0.495003,0.967033,0.028683,0.372385,0.001179,0.002538,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.dropna(inplace=True)

print(df.isna().sum())

Unnamed: 0                 0
AQ_pm10                    0
AQ_nh3                     0
AQ_nox                     0
AQ_no2                     0
WE_temp_2m                 0
WE_wind_speed_10m_mean     0
WE_wind_speed_10m_max      0
WE_tot_precipitation       0
WE_precipitation_t         0
WE_surface_pressure        0
WE_solar_radiation         0
WE_rh_min                  0
WE_rh_mean                 0
WE_rh_max                  0
WE_wind_speed_100m_mean    0
WE_wind_speed_100m_max     0
WE_blh_layer_max           0
WE_blh_layer_min           0
EM_nh3_livestock_mm        0
EM_nh3_agr_soils           0
EM_nh3_agr_waste_burn      0
EM_nh3_sum                 0
EM_nox_traffic             0
EM_nox_sum                 0
EM_so2_sum                 0
LI_pigs                    0
LI_bovine                  0
LI_pigs_v2                 0
LI_bovine_v2               0
LA_hvi                     0
LA_lvi                     0
LA_land_use                0
LA_soil_use                0
wind_dir_10m_N

# Regression using LSTM and Kfold

In [6]:
# Define the features (independent variables) and the target (dependent variable)
X = df.drop(columns=['AQ_pm10'])
y = df['AQ_pm10']

# Set the number of folds for cross-validation
num_folds = 10

# Initialize lists to store the metrics
mae_scores = []
mse_scores = []
rmse_scores = []
r2_scores = []

# Define the architecture of the LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(32, input_shape=(X.shape[1], 1), activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Perform k-fold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)

    model = create_model()

    # Train the model
    model.fit(X_train, y_train, epochs=400, batch_size=32, verbose=0)

    # Predict on the test data
    y_pred = model.predict(X_test)

    # Calculate metrics for the current fold
    fold_mae = mean_absolute_error(y_test, y_pred)
    fold_mse = mean_squared_error(y_test, y_pred)
    fold_rmse = math.sqrt(fold_mse)
    fold_r2 = r2_score(y_test, y_pred)

    # Add the metrics to the lists
    mae_scores.append(fold_mae)
    mse_scores.append(fold_mse)
    rmse_scores.append(fold_rmse)
    r2_scores.append(fold_r2)

# Calculate the averages of the metrics across all folds
avg_mae = np.mean(mae_scores)
avg_mse = np.mean(mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_r2 = np.mean(r2_scores)

print('Average MAE across', num_folds, 'folds:', round(avg_mae, 2))
print('Average MSE across', num_folds, 'folds:', round(avg_mse, 2))
print('Average RMSE across', num_folds, 'folds:', round(avg_rmse, 2))
print('Average R-squared (R2) across', num_folds, 'folds:', round(avg_r2, 2))

5/5 [==============================] - 0s 6ms/step
Average MAE across 10 folds: 0.41
Average MSE across 10 folds: 0.87
Average RMSE across 10 folds: 0.44
Average R-squared (R2) across 10 folds: -44.87
